In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
import tensorflow as tf
sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.initialize_all_variables())
y = tf.nn.softmax(tf.matmul(x,W) + b)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y),
    reduction_indices=[1]))

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import time

time0 = time.time()

for i in range(2000):

  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    # train_accuracy = accuracy.eval(feed_dict={
    #     x:batch[0], y_: batch[1], keep_prob: 1.0})
    # print("step %d, training accuracy %.5f"%(i, train_accuracy))
    print("step %d, test accuracy %.5f"%(i, accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %.5f"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, test accuracy 0.09560
step 100, test accuracy 0.81920
step 200, test accuracy 0.89150
step 300, test accuracy 0.91610
step 400, test accuracy 0.92970
step 500, test accuracy 0.93370
step 600, test accuracy 0.93930
step 700, test accuracy 0.94320
step 800, test accuracy 0.95140
step 900, test accuracy 0.95330
step 1000, test accuracy 0.95690
step 1100, test accuracy 0.96030
step 1200, test accuracy 0.96000
step 1300, test accuracy 0.96540
step 1400, test accuracy 0.96860
step 1500, test accuracy 0.96720
step 1600, test accuracy 0.96600
step 1700, test accuracy 0.96930
step 1800, test accuracy 0.97140
step 1900, test accuracy 0.97190
test accuracy 0.97410


In [5]:
optimizer=tf.train.AdamOptimizer(1e-4)

In [7]:
gradients=optimizer.compute_gradients(cross_entropy)

In [11]:
gradients

[(None, <tensorflow.python.ops.variables.Variable at 0x7f807c081d10>),
 (None, <tensorflow.python.ops.variables.Variable at 0x7f807c081c90>),
 (<tf.Tensor 'gradients_2/Conv2D_grad/tuple/control_dependency_1:0' shape=(5, 5, 1, 32) dtype=float32>,
  <tensorflow.python.ops.variables.Variable at 0x7f807c081f10>),
 (<tf.Tensor 'gradients_2/add_1_grad/tuple/control_dependency_1:0' shape=(32,) dtype=float32>,
  <tensorflow.python.ops.variables.Variable at 0x7f80740edb10>),
 (<tf.Tensor 'gradients_2/Conv2D_1_grad/tuple/control_dependency_1:0' shape=(5, 5, 32, 64) dtype=float32>,
  <tensorflow.python.ops.variables.Variable at 0x7f80740fe690>),
 (<tf.Tensor 'gradients_2/add_2_grad/tuple/control_dependency_1:0' shape=(64,) dtype=float32>,
  <tensorflow.python.ops.variables.Variable at 0x7f807410d350>),
 (<tf.Tensor 'gradients_2/MatMul_1_grad/tuple/control_dependency_1:0' shape=(3136, 1024) dtype=float32>,
  <tensorflow.python.ops.variables.Variable at 0x7f80740fef10>),
 (<tf.Tensor 'gradients_2/a

In [12]:
sess.graph

In [7]:
accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})

0.1135

In [6]:
mnist.test.images.__class__

numpy.ndarray